# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Store csv created in part one into a DataFrame
city_df = pd.read_csv("data/city_df.csv")
city_df.head()

,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp
0,Mataura,-46.19,168.86,80,31,7.60,NZ,1607837305,53.006
1,Hobart,-42.88,147.33,41,75,7.20,AU,1607837120,78.998
2,Kruisfontein,-34.00,24.73,93,20,1.92,ZA,1607837307,62.006
3,Lüderitz,-26.65,15.16,82,0,2.60,NaN,1607837320,57.200
4,Albany,42.60,-73.97,89,100,2.23,US,1607837308,50.000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [10]:
# Heatmap of humidity
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
figure_layout = {'width': '800px',
                'height': '400px',
                'border':'1px solid black',
                'padding': '1px'}
fig = gmaps.figure(layout = figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='800px'))

In [11]:
city_df.head(100)

,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp
0,Mataura,-46.19,168.86,80,31,7.60,NZ,1607837305,53.006
1,Hobart,-42.88,147.33,41,75,7.20,AU,1607837120,78.998
2,Kruisfontein,-34.00,24.73,93,20,1.92,ZA,1607837307,62.006
3,Lüderitz,-26.65,15.16,82,0,2.60,NaN,1607837320,57.200
4,Albany,42.60,-73.97,89,100,2.23,US,1607837308,50.000
...,...,...,...,...,...,...,...,...,...
95,Soubre,5.79,-6.61,98,30,1.00,CI,1607837603,71.528
96,Saint Lawrence,36.06,14.20,82,20,7.20,MT,1607837603,57.992
97,Esperance,-33.87,121.90,51,40,9.30,AU,1607837340,80.600
98,Neuquén,-38.95,-68.06,54,0,4.10,AR,1607837327,55.400


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
perfect_city = city_df.loc[(city_df['Cloudiness']==0)&(city_df['Wind Speed']<=10) & (city_df['Max Temp'] >= 70) & (city_df['Max Temp'] <= 80)].dropna()
# The cities having the ideal weather
perfect_city

,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp
61,Roura,4.73,-52.33,100,0,1.00,GF,1607837598,73.400
64,Departamento de Maldonado,-34.67,-54.92,94,0,1.50,UY,1607837598,71.006
118,Salalah,17.02,54.09,25,0,6.20,OM,1607837343,75.200
134,Alenquer,-1.94,-54.74,94,0,1.50,BR,1607837559,77.000
213,Cayenne,4.93,-52.33,100,0,1.00,GF,1607837343,73.400
216,Diplo,24.47,69.58,24,0,5.89,PK,1607837621,75.596
263,Boussé,12.66,-1.89,35,0,1.50,BF,1607837627,71.600
296,Cidreira,-30.18,-50.21,79,0,3.76,BR,1607837309,72.734
300,Várzea Grande,-15.65,-56.13,74,0,3.10,BR,1607837633,78.800
359,San Fernando del Valle de Catamarca,-28.47,-65.79,42,0,3.22,AR,1607837380,79.592


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = perfect_city

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
